In [88]:
import numpy as np
import pandas as pd
import os
from sklearn.preprocessing import MinMaxScaler

In [89]:
dirPath = './Data/'

In [90]:
datafiles = [e for e in os.listdir(dirPath) if os.path.isfile(os.path.join(dirPath, e)) ]

In [91]:
if '.DS_Store' in datafiles:
    datafiles.remove('.DS_Store')

In [92]:
xx = []
for s in datafiles:
    if 'BTC' in s:
        xx.append(s)

In [93]:
print(xx)

['BTC-USD.csv']


In [94]:
datafiles= xx

In [95]:
print((datafiles))

['BTC-USD.csv']


In [104]:
minDate = None
data_map = {}
for file in datafiles:
    scaler = MinMaxScaler()
    df = pd.read_csv(os.path.join(dirPath, file))
    df.dropna(how='any', axis=0, inplace=True)
    df = df.loc[9:, :]
    cols = list(df.columns)
    print(cols)
    cols.remove('Date')
    df[cols] = scaler.fit_transform(df[cols])
    if minDate is None:
        minDate = min(df['Date'])
    else:
        minDate = max(minDate, min(df['Date']))
    #minDate = min(df['Date'])
    df = df.drop(columns=['High', 'Low', 'Adj Close'])
    all_dates = pd.DataFrame(pd.date_range(df['Date'].min(), df['Date'].max()), columns=['Date'])
    #all_dates.Date.astype('datetime64[ns]')
    ndf = df.astype({'Date':'datetime64[ns]'})
    #print(ndf)
    df = all_dates.merge(right=ndf, how='left', on='Date')
    #print(df)
    df = df.fillna(method='ffill')
    colNames = df.columns
    key = file[:-4]
    colNames = [key+'_'+e if e != 'Date' else e for e in colNames]
    df.columns = colNames
    #print(df)
    #print(file)
    #print(minDate)
    print(df)
    data_map[key] = df
#print(minDate)

['Date', 'Open', 'High', 'Low', 'Close', 'Adj Close', 'Volume']
           Date  BTC-USD_Open  BTC-USD_Close  BTC-USD_Volume
0    2014-09-26      0.003481       0.003358        0.000044
1    2014-09-27      0.003364       0.003286        0.000026
2    2014-09-28      0.003304       0.002954        0.000050
3    2014-09-29      0.002969       0.002929        0.000076
4    2014-09-30      0.002957       0.003099        0.000082
...         ...           ...            ...             ...
2994 2022-12-07      0.251030       0.247371        0.056044
2995 2022-12-08      0.247436       0.253089        0.058384
2996 2022-12-09      0.253147       0.251601        0.057905
2997 2022-12-10      0.251694       0.251535        0.036189
2998 2022-12-11      0.251685       0.252082        0.036197

[2999 rows x 4 columns]


In [105]:
time_slices = None
for key, val in data_map.items():
    tmp = data_map[key]
    data_map[key] = tmp[tmp['Date'] >= minDate].sort_values(by=['Date']).reset_index(drop=True)
    if time_slices is None:
        time_slices = pd.DataFrame(data_map[key]['Date'],columns=['Date'])

In [106]:
mdf = None
for key, val in data_map.items():
    #print(val)
    if mdf is None:
        mdf = val
    else:
        mdf = mdf.merge(val, on='Date', how='inner')
mdf.head()

,Date,BTC-USD_Open,BTC-USD_Close,BTC-USD_Volume
0,2014-09-26,0.003481,0.003358,0.000044
1,2014-09-27,0.003364,0.003286,0.000026
2,2014-09-28,0.003304,0.002954,0.000050
3,2014-09-29,0.002969,0.002929,0.000076
4,2014-09-30,0.002957,0.003099,0.000082


In [107]:
mdf.tail()

,Date,BTC-USD_Open,BTC-USD_Close,BTC-USD_Volume
2994,2022-12-07,0.251030,0.247371,0.056044
2995,2022-12-08,0.247436,0.253089,0.058384
2996,2022-12-09,0.253147,0.251601,0.057905
2997,2022-12-10,0.251694,0.251535,0.036189
2998,2022-12-11,0.251685,0.252082,0.036197


In [108]:
mdf.dropna(how='any', axis=0, inplace=True) 

In [109]:
import pickle

In [11]:
f = open("data.pkl","wb")
pickle.dump(data_map,f)
f.close()

In [12]:
datafiles.sort()

In [110]:
filter_ = max(mdf['Date'])
time_slices = time_slices[time_slices['Date'] <= filter_]
time_slices.sort_values('Date', inplace=True)

In [111]:
time_slices

,Date
0,2014-09-26
1,2014-09-27
2,2014-09-28
3,2014-09-29
4,2014-09-30
...,...
2994,2022-12-07
2995,2022-12-08
2996,2022-12-09
2997,2022-12-10


In [112]:
import datetime
min_t, max_t = min(time_slices['Date']), max(time_slices['Date'])
min_t, max_t = min_t.to_pydatetime(), max_t.to_pydatetime()
#x = min_t + datetime.timedelta(days=1)
#min_t, max_t = datetime.datetime.strptime(min_t, '%Y-%m-%d').date(), \
#datetime.datetime.strptime(max_t, '%Y-%m-%d').date()
days = (max_t - min_t).days

In [113]:
print(max_t)

2022-12-11 00:00:00


In [115]:
print(days)
days=days+1

2998


In [116]:
max_close, min_close = max(mdf['BTC-USD_Close']), min(mdf['BTC-USD_Close'])

In [117]:
print(max_close, min_close)

1.0 0.0


Data prepared previously as 5x5x1840x3

In [63]:
allData = [[[None for _ in range(days)] for _ in range(5)] for _ in range(5)]

In [64]:
start = min_t.strftime('%Y-%m-%d')
for i in range(len(datafiles)):
    file = datafiles[i]
    fname = file[:-4]
    data = data_map[fname]
    r, c = i//5, i%5
    for j in range(days):
        curr = (min_t + datetime.timedelta(days=j)).strftime('%Y-%m-%d')
        tmp = data[data['Date'] == curr]
        tmp = tmp.drop(columns=['Date'])
        tt = tuple(tmp.to_records(index=False)[0])
        #print(type(tt))
        allData[r][c][j] = tt

In [65]:
n_files = np.array(datafiles)
n_data = np.array(allData, dtype='object')

In [66]:
n_data.shape

(5, 5, 1840, 3)

In [67]:
with open('stocks.npy', 'wb') as f:
    np.save(f, n_files)
with open('np_data_scaled.npy', 'wb') as f:
    np.save(f, n_data)

Data as 25x1840x3

Preparing data

In [118]:
mdf.tail(10)

,Date,BTC-USD_Open,BTC-USD_Close,BTC-USD_Volume
2989,2022-12-02,0.249237,0.250940,0.055658
2990,2022-12-03,0.251039,0.248263,0.046193
2991,2022-12-04,0.248339,0.251561,0.047921
2992,2022-12-05,0.251615,0.249251,0.063264
2993,2022-12-06,0.249334,0.250953,0.056656
2994,2022-12-07,0.251030,0.247371,0.056044
2995,2022-12-08,0.247436,0.253089,0.058384
2996,2022-12-09,0.253147,0.251601,0.057905
2997,2022-12-10,0.251694,0.251535,0.036189
2998,2022-12-11,0.251685,0.252082,0.036197


In [119]:
mdf.describe()

,BTC-USD_Open,BTC-USD_Close,BTC-USD_Volume
count,2999.000000,2999.000000,2999.000000
mean,0.188380,0.188378,0.046681
std,0.241206,0.241076,0.057122
min,0.000000,0.000000,0.000000
25%,0.007429,0.007415,0.000274
50%,0.101939,0.101943,0.020720
75%,0.245053,0.245242,0.080142
max,1.000000,1.000000,1.000000


In [120]:
#from datetime import datetime
from sklearn.preprocessing import MinMaxScaler

def prepareData(time_slices, mdf, n_steps=90):
    X = []
    y = []
    scaler = MinMaxScaler()
    mdf.Date = pd.to_datetime(mdf.Date)
    nn = mdf.drop(columns=['Date'])
    scaler.fit(nn)
    col_X, col_y = None, None
    max_time = max(mdf['Date']).to_pydatetime()
    min_time = min(mdf['Date']).to_pydatetime()
    dd = (max_time - min_time).days
    s = min_time
    #print(time_slices.shape)
    #for start in mdf['Date']:
    for i in range(dd-128):
        sample = []
        #s = start.to_pydatetime()
        #print(s)
        end = (s + datetime.timedelta(days=n_steps))
        #print(end)
        # check if we are beyond the sequence
        if end >= max_time:
            break
        
        tmp_X = mdf[mdf['Date'] >= s]
        tmp_X = tmp_X[tmp_X['Date'] < end]
        tmp_y = mdf[mdf['Date'] == end]
        #print(tmp_y)
        tmp_X = tmp_X.drop(columns = ['Date'])
        tmp_X_scaled = scaler.transform(tmp_X)
        #tmp_y = tmp_y['ETH-USD']
        if col_X is None:
            col_X = tmp_X.columns
            col_y = tmp_y.columns
        arr_tmp_X = tmp_X.to_numpy()
        arr_tmp_y = tmp_y.to_numpy()
        #print(arr_tmp_y.shape)
        for i in range(arr_tmp_X.shape[0]):
            sample.append(arr_tmp_X[i,:])
        X.append(np.array(sample))
        y.append(arr_tmp_y[0,:])
        s = (s + datetime.timedelta(days=1))
    #print(len(X))
    #return np.array(X), np.array(y)
    return np.array(X), pd.DataFrame(data=y, columns=col_y)

In [154]:
mdf.shape

(2999, 4)

In [155]:
size = int(0.8 * mdf.shape[0])
print(size)

2399


In [167]:
tmp = mdf.copy()
train_df = tmp.loc[:size,:]
val_df = tmp.loc[size:size+300,:]
val_df.reset_index(inplace=True, drop=True)
test_df = tmp.loc[size+300:,:]
test_df.reset_index(inplace=True, drop=True)

In [168]:
train_df.shape

(2400, 4)

In [169]:
val_df.shape

(301, 4)

In [170]:
test_df.shape

(300, 4)

In [171]:
test_df

,Date,BTC-USD_Open,BTC-USD_Close,BTC-USD_Volume
0,2022-02-15,0.629476,0.658821,0.064724
1,2022-02-16,0.659040,0.649719,0.056378
2,2022-02-17,0.649522,0.598912,0.074768
3,2022-02-18,0.599281,0.591388,0.066401
4,2022-02-19,0.591472,0.592741,0.039123
...,...,...,...,...
295,2022-12-07,0.251030,0.247371,0.056044
296,2022-12-08,0.247436,0.253089,0.058384
297,2022-12-09,0.253147,0.251601,0.057905
298,2022-12-10,0.251694,0.251535,0.036189


In [172]:
X_train, y_train = prepareData(time_slices, train_df, n_steps=128)
X_val, y_val = prepareData(time_slices, val_df, n_steps=128)
X_test, y_test = prepareData(time_slices, test_df, n_steps=128)

/var/folders/q3/804jvxv50k900sffyj_lk9yc0000gn/T/ipykernel_27756/1836777357.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  mdf.Date = pd.to_datetime(mdf.Date)
/var/folders/q3/804jvxv50k900sffyj_lk9yc0000gn/T/ipykernel_27756/1836777357.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  mdf.Date = pd.to_datetime(mdf.Date)
/var/folders/q3/804jvxv50k900sffyj_lk9yc0000gn/T/ipykernel_27756/1836777357.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using 

In [173]:
print(X_train.shape)

(2271, 128, 3)


In [178]:
y_train.loc[0,:]

Date              2015-02-01 00:00:00
BTC-USD_Open                 0.000593
BTC-USD_Close                0.000725
BTC-USD_Volume               0.000066
Name: 0, dtype: object

In [174]:
print(y_train.shape)

(2271, 4)


In [175]:
print(X_val.shape, X_test.shape)

(172, 128, 3) (171, 128, 3)


In [179]:
print(y_val.shape, y_test.shape)

(172, 4) (171, 4)


In [184]:
print(X_train[:,:,1].shape)

(2271, 128)


In [176]:
f = open("data_X_train.pkl","wb")
pickle.dump(X_train,f)
f.close()

f = open("data_X_val.pkl","wb")
pickle.dump(X_val,f)
f.close()

f = open("data_X_test.pkl","wb")
pickle.dump(X_test,f)
f.close()

y_train.to_pickle("./data_y_train.pkl")

y_val.to_pickle("./data_y_val.pkl") 

y_test.to_pickle("./data_y_test.pkl") 

In [51]:
f = open("data_X_new3.pkl","wb")
pickle.dump(X,f)
f.close()

y.to_pickle("./data_y_new3.pkl") 

In [100]:
tmp = pd.read_csv('./Data/BTC-USD.csv')

In [101]:
tmp.dropna(how='any', axis=0, inplace=True)

In [102]:
tmp.shape

(3008, 7)